In [22]:
import random
from datetime import datetime, timedelta, timezone
import firebase_admin
from firebase_admin import credentials, firestore

# -------------------------------
# Firebase initialization
# -------------------------------
if not firebase_admin._apps:
    cred = credentials.Certificate("track-pit-firebase-adminsdk-fbsvc-989e750375.json")
    firebase_admin.initialize_app(cred)

db = firestore.client()

uid = "iiYYy1LD2TYMASxgaW1VzTvRkwe2"
vehicleId = "TtkkQVThcYodgjnSizKb"  # Proton X50 only

now = datetime.now(timezone.utc)

# -------------------------------
# Step 1: Clear old services for this car
# -------------------------------
services_ref = db.collection("users").document(uid).collection("services")
old_services = services_ref.stream()

for doc in old_services:
    data = doc.to_dict()
    if data.get("vehicleId") == vehicleId:
        # delete subcollection statusTimeline first
        timeline_ref = services_ref.document(doc.id).collection("statusTimeline")
        for st_doc in timeline_ref.stream():
            st_doc.reference.delete()
        # delete service doc
        services_ref.document(doc.id).delete()
        print(f"🗑️ Deleted old service {doc.id}")

# -------------------------------
# Helpers for new mock data
# -------------------------------
def random_booking_time(base_day):
    """Return a booking datetime snapped to 30-min slot between 8:00–19:00"""
    base = base_day.replace(hour=8, minute=0, second=0, microsecond=0)
    slots = 22  # 8:00 → 19:00 = 22 slots
    slot = random.randint(0, slots)
    return base + timedelta(minutes=30*slot)

def create_timeline(status_type, booked_time):
    statuses = ["inspection","parts_awaiting","in_repair","completed"]

    if status_type == "upcoming":
        return [{"status": s, "completedAt": None} for s in statuses]

    elif status_type == "ongoing":
        cut = random.randint(1, 2)  # finished 1–2 steps
        result = []
        for i, s in enumerate(statuses):
            if i < cut:
                result.append({"status": s, "completedAt": booked_time + timedelta(hours=2*i)})
            else:
                result.append({"status": s, "completedAt": None})
        return result

    elif status_type == "completed":
        return [{"status": s, "completedAt": booked_time + timedelta(hours=2*i)} for i, s in enumerate(statuses)]

# -------------------------------
# Step 2: Insert new 7 services
# -------------------------------
mock_plan = (
    ["upcoming"] * 2 +
    ["ongoing"] * 3 +
    ["completed"] * 2
)

for status_type in mock_plan:
    if status_type == "upcoming":
        day = now + timedelta(days=random.randint(0, 3))  # today → next 3 days
    elif status_type == "ongoing":
        day = now - timedelta(days=random.randint(0, 1))  # today or yesterday
    elif status_type == "completed":
        day = now - timedelta(days=random.randint(1, 3))  # 1–3 days ago

    booked_time = random_booking_time(day)

    # ensure createdAt is 1–3 days before booked_time
    created_time = booked_time - timedelta(days=random.randint(1, 3))

    service_doc = {
        "vehicleId": vehicleId,
        "serviceTypeId": random.randint(1, 3),
        "workshopId": random.randint(1, 2),
        "bookedDateTime": booked_time,
        "notes": f"{status_type.title()} mock service",
        "createdAt": created_time,
        "updatedAt": created_time,  # always same as createdAt
    }

    service_ref = services_ref.document()
    service_ref.set(service_doc)

    timeline_ref = service_ref.collection("statusTimeline")
    for idx, st in enumerate(create_timeline(status_type, booked_time), start=1):
        timeline_ref.document(str(idx)).set(st)

    print(f"✅ Added {status_type} service {service_ref.id} at {booked_time}, created {created_time}")

🗑️ Deleted old service 4cekJrziNHQ4jnIVkFsb
🗑️ Deleted old service 5nVnTboxoi3Hx5ari67A
🗑️ Deleted old service TIpxrG0p8Pva06SYSUyI
🗑️ Deleted old service b8cCJSJAl5i1wpVY3ti4
🗑️ Deleted old service bMAy8auakt4ZKRiL1LoW
🗑️ Deleted old service cwLEKK0b59dUmROxhomk
🗑️ Deleted old service dPsnD0JT8Hxm68vHUajx
🗑️ Deleted old service iXnvxGyK4lBkf67WxH5A
✅ Added upcoming service SvUYOiQBJXxtVmIGhho2 at 2025-09-22 09:30:00+00:00, created 2025-09-20 09:30:00+00:00
✅ Added upcoming service nxlK2Ud7NSs6ANfmO6Cf at 2025-09-20 08:00:00+00:00, created 2025-09-17 08:00:00+00:00
✅ Added ongoing service iwzx8rcCdgeOSF1pzs4r at 2025-09-18 17:00:00+00:00, created 2025-09-15 17:00:00+00:00
✅ Added ongoing service 5O6aKTw2TXroqg0BGrwy at 2025-09-18 10:30:00+00:00, created 2025-09-15 10:30:00+00:00
✅ Added ongoing service I35sCa9MRPJoCMw7c5nW at 2025-09-19 12:00:00+00:00, created 2025-09-16 12:00:00+00:00
✅ Added completed service ZXfJSPe442AL2lDgtKrf at 2025-09-16 13:00:00+00:00, created 2025-09-14 13:00